In [1]:
import math
import numpy as np
from scipy.stats import norm
from chap8 import tree_lay, post_process, TreeNode

# 第七次作业-期货1-二叉树定价

In [2]:
# 二叉树定价
r = 0.03
u = 1.1
d = 0.9
S = 10
K = 10.5
q = 0.7
T = 3
p = (1 + r - d)/(u - d) #风险中性概率
sp_level = [2]  # 派息周期
sp_r = 0.1 #派息比例
root = TreeNode(S)
res = tree_lay(root, u, d, T, sp_level, sp_r)
print('无派息期权二叉树:{}'.format(res))

print('==============欧式期权情况==============')
C = [max(i - K, 0) for i in res[-1]][::-1] #从大到小排序，看涨期权
P = [max(K - i, 0) for i in res[-1]][::-1]#从大到小排序，看跌期权
print('看涨期权第{}周期价值：{}\n看跌期权第{}周期价值：{}'.format(T, [np.round(i, 3) for i in C], T, [np.round(i, 3) for i in P]))
ans_C, ans_P = 0, 0
for i in range(T+1):
    ans_C += (p ** (T - i)) * ((1 - p) ** (i)) * math.comb(T, i) * C[i] / (1 + r) ** T
for i in range(T+1):
    ans_P += (p ** (T - i)) * ((1 - p) ** (i)) * math.comb(T, i) * P[i] / (1 + r) ** T
print('看涨期权价格:{:.3f}\n看跌期权价格:{:.3f}'.format(ans_C, ans_P))

无派息期权二叉树:[[10], [9.0, 11.0], [8.1, 9.9, 12.1], [7.29, 8.91, 10.89, 13.31]]
==============欧式期权情况==============
看涨期权第3周期价值：[2.81, 0.39, 0, 0]
看跌期权第3周期价值：[0, 0, 1.59, 3.21]
看涨期权价格:0.865
看跌期权价格:0.474


In [3]:
post_process(res, K, r, p, sp_level, sp_r, mode='P')

====================START====================
-------看跌期权，第2周期-------
当前价值: [2.4000000000000004, 0.5999999999999996, 0]
风险中性下期望贴现: [2.093251015864132, 0.540052939419745, 0.0]
较高项: [2.4, 0.6, 0.0]
-------看跌期权，第1周期-------
当前价值: [1.5, 0]
风险中性下期望贴现: [1.1936480062646653, 0.2037935620451867]
较高项: [1.5, 0.204]
-------看跌期权，第0周期-------
当前价值: [0.5]
风险中性下期望贴现: [0.6380347639472169]
较高项: [0.638]
美式看跌期权定价:0.638
====================END====================



[[0.638], [1.5, 0.204], [2.4, 0.6, 0.0]]

# 第八次作业--期货2-BS公式

In [4]:
# 连续时间下的风险中性概率
sigma = 0.38
r = 0.055 #无风险利率
n = 40 #40期二叉树
T_t = 2/12 #多少个月到期/12（年）
t_delta = T_t / n
u = np.exp(sigma * np.sqrt(t_delta))
d = np.exp(-sigma * np.sqrt(t_delta))
q =0.5 - np.sqrt(t_delta) * (sigma / 4 - r / (2 * sigma))
print(q)

0.4985391378606411


In [36]:
# BS公式

def BS(S, K, t, sigma, r, mu=None, sp=None):
    d1 = 1 / (sigma * np.sqrt(t)) * (np.log(S / K) + (r + sigma**2 / 2) * t)
    d2 = d1 - sigma * np.sqrt(t)
    C = S * norm.cdf(d1) - K * np.exp(-r * t) * norm.cdf(d2)
    D = K * np.exp(-r * t) * norm.cdf(-d2) - S * norm.cdf(-d1)
    if mu:
        tmp = (np.log(K) - (np.log(S) + (mu - sigma**2 / 2) * t )) / (sigma * np.sqrt(t))
        prob = norm.cdf(tmp)
        print('看涨被执行概率: {:.3f}'.format(1 - prob))
        print('看跌被执行概率: {:.3f}'.format(prob))
        if sp:
            tmp_sp = (np.log(sp) - (np.log(S) + (mu - sigma**2 / 2) * t )) / (sigma * np.sqrt(t))
            prob_sp = norm.cdf(tmp_sp)
            print('S(t)<sp概率: {:.3f}'.format(prob_sp))
    print('d1={:.3f}, d2={:.3f}\n看涨期权价格: {:.3f}\n看跌期权价格: {:.3f}'.format(d1, d2, C, D))

T_t = 6/12 # 月份/12（年）
sigma = 0.3 #年波动率
r = 0.08 #无风险利率
K = 42 #协议价格
S = 40 #标的资产当前价格
mu = r
sp = 0.9 * S
#S = 30 - np.exp(-r * 1.5/12) #标的资产当前价格-派息贴现
BS(S, K, T_t, sigma, r, mu, sp)

看涨被执行概率: 0.441
看跌被执行概率: 0.559
S(t)<sp概率: 0.281
d1=0.065, d2=-0.148
看涨期权价格: 3.220
看跌期权价格: 3.573


In [19]:
T_t = 5/12
sigma = 0.2
r = 0.06
K = 20
S = 19 * np.exp(-r * T_t)
print('S: {:.3f}'.format(S))
BS(S, K, T_t, sigma, r)

S: 18.531
d1: -0.333, d2: -0.462, 看涨期权价格: 0.567, 看跌期权价格: 1.543


In [44]:
T_t = 1/2
sigma = 0.3
r = 0.01
K = 1
S = 0.2
mu = 0.08
sp = 0.9 * S
print('S: {:.3f}'.format(S))
BS(S, K, T_t, sigma, r, mu, sp)

S: 0.200
看涨被执行概率: 0.000
看跌被执行概率: 1.000
S(t)<sp概率: 0.281
d1: -7.457, d2: -7.669, 看涨期权价格: 0.000, 看跌期权价格: 0.795


26.998183240180282